In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
#offertimes.to_csv('offertimes.csv')
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
#inc_prod_num.to_csv('inc_prod_num.csv')
choice_prob = raw_jd_choice['clicknum'] / raw_jd_choice.groupby('clickset')['clicknum'].transform('sum')
raw_jd_choice['choice prob'] = choice_prob

assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

# all purchase options in each assortment
purchaseset = raw_jd_choice['purchaseset']
purchaseset_list = []
for pset in purchaseset:
    num_lst = json.loads(pset)
    purchaseset_list.append(num_lst)
raw_jd_choice['purchaseset'] = purchaseset_list

purchasenum_list = raw_jd_choice['clicknum'].tolist()
jd_choice=pd.DataFrame({'clickset':clickset_list,'purchaseset':purchaseset_list,'purchasenum':purchasenum_list,'choice prob':choice_prob})

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
collection = []
for cset in clickset_list:
    if cset not in collection:
        collection.append(cset)
print('there are {} different assortments'.format(len(collection)))

assortment_info_df['assortments'] = collection
choice_collection = np.zeros((n,len(collection)))
global_num = 0
for i in range(len(collection)):
    local_num =0
    while(local_num < inc_prod_num[i]):
        if purchaseset_list[global_num][0] in collection[i]:
            choice_collection[purchaseset_list[global_num][0]][i] = choice_prob[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print('error!')
full_choice_collection = pd.DataFrame(choice_collection)
#full_choice_collection.to_csv('instances/full_instances/full_choice.csv')


there are 9 different products
there are 134 different assortments


In [4]:
assortment_info_df_20 = assortment_info_df[(assortment_info_df['offer_times']>=20)]
assortments_20 = []
for i in range(len(assortment_info_df_20['assortments'])):
    assortments_20.append(assortment_info_df_20['assortments'][i])


jd_choice_20 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_20:
        jd_choice_20.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_20 = assortment_info_df_20['assortments']
choice_collection_20 = np.zeros((n,len(collection_20)))

frequency_collection_20 = np.zeros((n,len(collection_20)))

purchaseset_list_20 = jd_choice_20['purchaseset'].tolist()

inc_prod_num_20 = assortment_info_df_20['includ_prod_num']
choice_prob_20 = jd_choice_20['choice prob']
frequency_20 = jd_choice_20['purchasenum']

global_num = 0
for i in range(len(collection_20)):
    local_num =0
    while(local_num < inc_prod_num_20[i]):
        if purchaseset_list_20[global_num][0] in assortments_20[i]:
            choice_collection_20[purchaseset_list_20[global_num][0]][i] = choice_prob_20[global_num]
            frequency_collection_20[purchaseset_list_20[global_num][0]][i] = frequency_20[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_20[global_num])
            print(assortments_20[i])
            print('error!')
            break
#print(global_num)  

choice_check_20 = pd.DataFrame(choice_collection_20)
#choice_check_20.to_csv('instances/full_instances/full_20.csv')
frequency_collection_20 = frequency_collection_20.astype(int)
frequency_check_20 = pd.DataFrame(frequency_collection_20)
#frequency_check_20.to_csv('instances/full_instances/frequency_20.csv')

offer_times_20 = assortment_info_df_20['offer_times'].tolist()
print('the number of assortments with at least 20 purchases is',len(assortments_20))

the number of assortments with at least 20 purchases is 29


In [5]:
assortment_info_df_30 = assortment_info_df[(assortment_info_df['offer_times']>=30)]
assortments_30 = []
for i in range(len(assortment_info_df_30['assortments'])):
    assortments_30.append(assortment_info_df_30['assortments'][i])


jd_choice_30 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_30:
        jd_choice_30.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_30 = assortment_info_df_30['assortments']
choice_collection_30 = np.zeros((n,len(collection_30)))

frequency_collection_30 = np.zeros((n,len(collection_30)))

purchaseset_list_30 = jd_choice_30['purchaseset'].tolist()

inc_prod_num_30 = assortment_info_df_30['includ_prod_num']
choice_prob_30 = jd_choice_30['choice prob']
frequency_30 = jd_choice_30['purchasenum']

global_num = 0
for i in range(len(collection_30)):
    local_num =0
    while(local_num < inc_prod_num_30[i]):
        if purchaseset_list_30[global_num][0] in assortments_30[i]:
            choice_collection_30[purchaseset_list_30[global_num][0]][i] = choice_prob_30[global_num]
            frequency_collection_30[purchaseset_list_30[global_num][0]][i] = frequency_30[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_30[global_num])
            print(assortments_30[i])
            print('error!')
            break
#print(global_num)  

choice_check_30 = pd.DataFrame(choice_collection_30)
#choice_check_30.to_csv('instances/full_instances/full_30.csv')
frequency_collection_30 = frequency_collection_30.astype(int)
frequency_check_30 = pd.DataFrame(frequency_collection_30)
#frequency_check_30.to_csv('instances/full_instances/frequency_30.csv')

offer_times_30 = assortment_info_df_30['offer_times'].tolist()
print('the number of assortments with at least 30 purchases is',len(assortments_30))

the number of assortments with at least 30 purchases is 24


In [6]:
assortment_info_df_40 = assortment_info_df[(assortment_info_df['offer_times']>=40)]
assortments_40 = []
for i in range(len(assortment_info_df_40['assortments'])):
    assortments_40.append(assortment_info_df_40['assortments'][i])


jd_choice_40 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_40:
        jd_choice_40.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_40 = assortment_info_df_40['assortments']
choice_collection_40 = np.zeros((n,len(collection_40)))

frequency_collection_40 = np.zeros((n,len(collection_40)))

purchaseset_list_40 = jd_choice_40['purchaseset'].tolist()

inc_prod_num_40 = assortment_info_df_40['includ_prod_num']
choice_prob_40 = jd_choice_40['choice prob']
frequency_40 = jd_choice_40['purchasenum']

global_num = 0
for i in range(len(collection_40)):
    local_num =0
    while(local_num < inc_prod_num_40[i]):
        if purchaseset_list_40[global_num][0] in assortments_40[i]:
            choice_collection_40[purchaseset_list_40[global_num][0]][i] = choice_prob_40[global_num]
            frequency_collection_40[purchaseset_list_40[global_num][0]][i] = frequency_40[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_40[global_num])
            print(assortments_40[i])
            print('error!')
            break
#print(global_num)  

choice_check_40 = pd.DataFrame(choice_collection_40)
#choice_check_40.to_csv('instances/full_instances/full_40.csv')
frequency_collection_40 = frequency_collection_40.astype(int)
frequency_check_40 = pd.DataFrame(frequency_collection_40)
#frequency_check_40.to_csv('instances/full_instances/frequency_40.csv')

offer_times_40 = assortment_info_df_40['offer_times'].tolist()
print('the number of assortments with at least 40 purchases is',len(assortments_40))

the number of assortments with at least 40 purchases is 19


In [7]:
assortment_info_df_50 = assortment_info_df[(assortment_info_df['offer_times']>=50)]
assortments_50 = []
for i in range(len(assortment_info_df_50['assortments'])):
    assortments_50.append(assortment_info_df_50['assortments'][i])


jd_choice_50 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_50:
        jd_choice_50.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_50 = assortment_info_df_50['assortments']
choice_collection_50 = np.zeros((n,len(collection_50)))

frequency_collection_50 = np.zeros((n,len(collection_50)))

purchaseset_list_50 = jd_choice_50['purchaseset'].tolist()

inc_prod_num_50 = assortment_info_df_50['includ_prod_num']
choice_prob_50 = jd_choice_50['choice prob']
frequency_50 = jd_choice_50['purchasenum']

global_num = 0
for i in range(len(collection_50)):
    local_num =0
    while(local_num < inc_prod_num_50[i]):
        if purchaseset_list_50[global_num][0] in assortments_50[i]:
            choice_collection_50[purchaseset_list_50[global_num][0]][i] = choice_prob_50[global_num]
            frequency_collection_50[purchaseset_list_50[global_num][0]][i] = frequency_50[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_50[global_num])
            print(assortments_50[i])
            print('error!')
            break
#print(global_num)  

choice_check_50 = pd.DataFrame(choice_collection_50)
#choice_check_50.to_csv('instances/full_instances/full_50.csv')
frequency_collection_50 = frequency_collection_50.astype(int)
frequency_check_50 = pd.DataFrame(frequency_collection_50)
#frequency_check_50.to_csv('instances/full_instances/frequency_50.csv')

offer_times_50 = assortment_info_df_50['offer_times'].tolist()
print('the number of assortments with at least 50 purchases is',len(assortments_50))

the number of assortments with at least 50 purchases is 15


In [8]:
assortment_info_df_60 = assortment_info_df[(assortment_info_df['offer_times']>=60)]
assortments_60 = []
for i in range(len(assortment_info_df_60['assortments'])):
    assortments_60.append(assortment_info_df_60['assortments'][i])


jd_choice_60 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_60:
        jd_choice_60.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_60 = assortment_info_df_60['assortments']
choice_collection_60 = np.zeros((n,len(collection_60)))

frequency_collection_60 = np.zeros((n,len(collection_60)))

purchaseset_list_60 = jd_choice_60['purchaseset'].tolist()

inc_prod_num_60 = assortment_info_df_60['includ_prod_num']
choice_prob_60 = jd_choice_60['choice prob']
frequency_60 = jd_choice_60['purchasenum']

global_num = 0
for i in range(len(collection_60)):
    local_num =0
    while(local_num < inc_prod_num_60[i]):
        if purchaseset_list_60[global_num][0] in assortments_60[i]:
            choice_collection_60[purchaseset_list_60[global_num][0]][i] = choice_prob_60[global_num]
            frequency_collection_60[purchaseset_list_60[global_num][0]][i] = frequency_60[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_60[global_num])
            print(assortments_60[i])
            print('error!')
            break
#print(global_num)  

choice_check_60 = pd.DataFrame(choice_collection_60)
#choice_check_60.to_csv('instances/full_instances/full_60.csv')
frequency_collection_60 = frequency_collection_60.astype(int)
frequency_check_60 = pd.DataFrame(frequency_collection_60)
#frequency_check_60.to_csv('instances/full_instances/frequency_60.csv')

offer_times_60 = assortment_info_df_60['offer_times'].tolist()
print('the number of assortments with at least 60 purchases is',len(assortments_60))

the number of assortments with at least 60 purchases is 13


In [9]:
assortment_info_df_75 = assortment_info_df[(assortment_info_df['offer_times']>=75)]
assortments_75 = []
for i in range(len(assortment_info_df_75['assortments'])):
    assortments_75.append(assortment_info_df_75['assortments'][i])


jd_choice_75 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_75:
        jd_choice_75.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_75 = assortment_info_df_75['assortments']
choice_collection_75 = np.zeros((n,len(collection_75)))

frequency_collection_75 = np.zeros((n,len(collection_75)))

purchaseset_list_75 = jd_choice_75['purchaseset'].tolist()

inc_prod_num_75 = assortment_info_df_75['includ_prod_num']
choice_prob_75 = jd_choice_75['choice prob']
frequency_75 = jd_choice_75['purchasenum']

global_num = 0
for i in range(len(collection_75)):
    local_num =0
    while(local_num < inc_prod_num_75[i]):
        if purchaseset_list_75[global_num][0] in assortments_75[i]:
            choice_collection_75[purchaseset_list_75[global_num][0]][i] = choice_prob_75[global_num]
            frequency_collection_75[purchaseset_list_75[global_num][0]][i] = frequency_75[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_75[global_num])
            print(assortments_75[i])
            print('error!')
            break
#print(global_num)  

choice_check_75 = pd.DataFrame(choice_collection_75)
#choice_check_75.to_csv('instances/full_instances/full_75.csv')
frequency_collection_75 = frequency_collection_75.astype(int)
frequency_check_75 = pd.DataFrame(frequency_collection_75)
#frequency_check_75.to_csv('instances/full_instances/frequency_75.csv')

offer_times_75 = assortment_info_df_75['offer_times'].tolist()
print('the number of assortments with at least 75 purchases is',len(assortments_75))

the number of assortments with at least 75 purchases is 12


In [10]:
assortment_info_df_100 = assortment_info_df[(assortment_info_df['offer_times']>=100)]
assortments_100 = []
for i in range(len(assortment_info_df_100['assortments'])):
    assortments_100.append(assortment_info_df_100['assortments'][i])

jd_choice_100 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_100:
        jd_choice_100.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_100 = assortment_info_df_100['assortments']
choice_collection_100 = np.zeros((n,len(collection_100)))

frequency_collection_100 = np.zeros((n,len(collection_100)))

purchaseset_list_100 = jd_choice_100['purchaseset'].tolist()

inc_prod_num_100 = assortment_info_df_100['includ_prod_num']
choice_prob_100 = jd_choice_100['choice prob']
frequency_100 = jd_choice_100['purchasenum']

global_num=0
for i in range(len(collection_100)):
    local_num =0
    while(local_num < inc_prod_num_100[i]):
        if purchaseset_list_100[global_num][0] in assortments_100[i]:
            choice_collection_100[purchaseset_list_100[global_num][0]][i] = choice_prob_100[global_num]
            frequency_collection_100[purchaseset_list_100[global_num][0]][i] = frequency_100[global_num]
            global_num = global_num +1
            local_num = local_num +1

choice_check_100 = pd.DataFrame(choice_collection_100)
#choice_check_100.to_csv('instances/full_instances/full_100.csv')
frequency_collection_100 = frequency_collection_100.astype(int)
frequency_check_100 = pd.DataFrame(frequency_collection_100)
#frequency_check_100.to_csv('instances/full_instances/frequency_100.csv')

offer_times_100 = assortment_info_df_100['offer_times'].tolist()
print('the number of assortments with at least 100 purchases is',len(assortments_100))

the number of assortments with at least 100 purchases is 11


compute sqrt(p_{i,S}(1-p_{i,S})/n_S)

In [33]:
quantity_20 = choice_collection_20/np.array(offer_times_20)
df_quantity_20 = pd.DataFrame(quantity_20)
df_quantity_20.to_csv('df_quantity/quantity_20.csv')
percent_20 = np.divide(quantity_20, choice_collection_20, where=choice_collection_20!=0 )
percent_20 [ choice_collection_20 ==0 ] = 0
df_percent_20 = pd.DataFrame(percent_20)
df_percent_20.to_csv('df_quantity/percent_20.csv')
max_nonzero_quanity_20 = np.max(quantity_20)
max_nonzero_percent_20 = np.max(percent_20)*100
print('max quantity with offertimes at least 20',max_nonzero_quanity_20)
print(f'max percentage with offertimes at least 20 {max_nonzero_percent_20} %')

max quantity with offertimes at least 20 0.03298611111111111
max percentage with offertimes at least 20 4.347826086956522 %


In [34]:
quantity_30 = choice_collection_30/np.array(offer_times_30)
df_quantity_30 = pd.DataFrame(quantity_30)
df_quantity_30.to_csv('df_quantity/quantity_30.csv')
percent_30 = np.divide(quantity_30, choice_collection_30, where=choice_collection_30!=0 )
percent_30 [ choice_collection_30 ==0 ] = 0
df_percent_30 = pd.DataFrame(percent_30)
df_percent_30.to_csv('df_quantity/percent_30.csv')
max_nonzero_quanity_30 = np.max(quantity_30)
max_nonzero_percent_30 = np.max(percent_30)*100
print('max quantity with offertimes at least 30',max_nonzero_quanity_30)
print(f'max percentage with offertimes at least 30 {max_nonzero_percent_30} %')

max quantity with offertimes at least 30 0.0244140625
max percentage with offertimes at least 30 3.125 %


In [35]:
quantity_40 = choice_collection_40/np.array(offer_times_40)
df_quantity_40 = pd.DataFrame(quantity_40)
df_quantity_40.to_csv('df_quantity/quantity_40.csv')
percent_40 = np.divide(quantity_40, choice_collection_40, where=choice_collection_40!=0 )
percent_40 [ choice_collection_40 ==0 ] = 0
df_percent_40 = pd.DataFrame(percent_40)
df_percent_40.to_csv('df_quantity/percent_40.csv')
max_nonzero_quanity_40 = np.max(quantity_40)
max_nonzero_percent_40 = np.max(percent_40)*100
print('max quantity with offertimes at least 40',max_nonzero_quanity_40)
print(f'max percentage with offertimes at least 40 {max_nonzero_percent_40} %')

max quantity with offertimes at least 40 0.01875
max percentage with offertimes at least 40 2.5 %


In [36]:
quantity_50 = choice_collection_50/np.array(offer_times_50)
df_quantity_50 = pd.DataFrame(quantity_50)
df_quantity_50.to_csv('df_quantity/quantity_50.csv')
percent_50 = np.divide(quantity_50, choice_collection_50, where=choice_collection_50!=0 )
percent_50 [ choice_collection_50 ==0 ] = 0
df_percent_50 = pd.DataFrame(percent_50)
df_percent_50.to_csv('df_quantity/percent_50.csv')
max_nonzero_quanity_50 = np.max(quantity_50)
max_nonzero_percent_50 = np.max(percent_50)*100
print('max quantity with offertimes at least 50',max_nonzero_quanity_50)
print(f'max percentage with offertimes at least 50 {max_nonzero_percent_50} %')

max quantity with offertimes at least 50 0.014609765474817378
max percentage with offertimes at least 50 1.9607843137254901 %


In [37]:
quantity_60 = choice_collection_60/np.array(offer_times_60)
df_quantity_60 = pd.DataFrame(quantity_60)
df_quantity_60.to_csv('df_quantity/quantity_60.csv')
percent_60 = np.divide(quantity_60, choice_collection_60, where=choice_collection_60!=0 )
percent_60 [ choice_collection_60 ==0 ] = 0
df_percent_60 = pd.DataFrame(percent_60)
df_percent_60.to_csv('df_quantity/percent_60.csv')
max_nonzero_quanity_60 = np.max(quantity_60)
max_nonzero_percent_60 = np.max(percent_60)*100
print('max quantity with offertimes at least 60',max_nonzero_quanity_60)
print(f'max percentage with offertimes at least 60 {max_nonzero_percent_60} %')

max quantity with offertimes at least 60 0.010247271107150813
max percentage with offertimes at least 60 1.4925373134328357 %


In [38]:
quantity_75 = choice_collection_75/np.array(offer_times_75)
df_quantity_75 = pd.DataFrame(quantity_75)
df_quantity_75.to_csv('df_quantity/quantity_75.csv')
percent_75 = np.divide(quantity_75, choice_collection_75, where=choice_collection_75!=0 )
percent_75 [ choice_collection_75 ==0 ] = 0
df_percent_75 = pd.DataFrame(percent_75)
df_percent_75.to_csv('df_quantity/percent_75.csv')
max_nonzero_quanity_75 = np.max(quantity_75)
max_nonzero_percent_75 = np.max(percent_75)*100
print('max quantity with offertimes at least 75',max_nonzero_quanity_75)
print(f'max percentage with offertimes at least 75 {max_nonzero_percent_75} %')

max quantity with offertimes at least 75 0.009040105193951348
max percentage with offertimes at least 75 1.282051282051282 %


In [39]:
quantity_100 = choice_collection_100/np.array(offer_times_100)
df_quantity_100 = pd.DataFrame(quantity_100)
df_quantity_100.to_csv('df_quantity/quantity_100.csv')
percent_100 = np.divide(quantity_100, choice_collection_100, where=choice_collection_100!=0 )
percent_100 [ choice_collection_100 ==0 ] = 0
df_percent_100 = pd.DataFrame(percent_100)
df_percent_100.to_csv('df_quantity/percent_100.csv')
max_nonzero_quanity_100 = np.max(quantity_100)
max_nonzero_percent_100 = np.max(percent_100)*100
print('max quantity with offertimes at least 100',max_nonzero_quanity_100)
print(f'max percentage with offertimes at least 100 {max_nonzero_percent_100} %')

max quantity with offertimes at least 100 0.006828824141519251
max percentage with offertimes at least 100 0.8130081300813009 %


In [51]:
offertimes_lst = [20,30,40,50,60,75,100]
collection_size = [len(assortments_20),len(assortments_30),len(assortments_40), len(assortments_50),len(assortments_60),len(assortments_75),len(assortments_100)]
max_quantity_lst = [max_nonzero_quanity_20,max_nonzero_quanity_30,max_nonzero_quanity_40,max_nonzero_quanity_50,max_nonzero_quanity_60,max_nonzero_quanity_75,max_nonzero_quanity_100]
max_percent_lst = [max_nonzero_percent_20,max_nonzero_percent_30,max_nonzero_percent_40,max_nonzero_percent_50,max_nonzero_percent_60,max_nonzero_percent_75,max_nonzero_percent_100]

In [54]:
uncentainty_summary = pd.DataFrame({'offertimes':offertimes_lst,'collection_size':collection_size,'max_quanity':max_quantity_lst,'max_percentage(%)':max_percent_lst})
uncentainty_summary.to_csv('uncentainty_summary.csv')
uncentainty_summary

,offertimes,collection_size,max_quanity,max_percentage(%)
0,20,29,0.032986,4.347826
1,30,24,0.024414,3.125000
2,40,19,0.018750,2.500000
3,50,15,0.014610,1.960784
4,60,13,0.010247,1.492537
5,75,12,0.009040,1.282051
6,100,11,0.006829,0.813008
